In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
np.random.seed(42)
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import nltk, string, re, spacy,unicodedata, random
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
### Pretrained Word Embeddings
dataset_path = "/content/drive/MyDrive/Thesis Dataset/df1.csv"
folder_path = "/content/drive/MyDrive/Thesis Dataset"

CPU times: user 35.1 ms, sys: 3.25 ms, total: 38.4 ms
Wall time: 88.6 ms


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
!pip install xlrd==1.2.0
!pip install openpyxl
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: xlrd
    Found existing installation: xlrd 2.0.1
    Uninstalling xlrd-2.0.1:
      Successfully uninstalled xlrd-2.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.2 MB/s eta 0:00:00


In [6]:
import pandas as pd
from pandas import read_excel
import numpy as np
import re
from re import sub
import multiprocessing
from unidecode import unidecode
import os
from time import time
from collections import Counter

from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM,Dense,Dropout,Activation,Embedding,Flatten,Bidirectional,MaxPooling1D, Conv1D, GlobalMaxPooling1D, MaxPooling1D, Reshape
from tensorflow.keras.models import Sequential

from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


import h5py
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score


from nltk.corpus import stopwords
import nltk
import re
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

In [8]:
dataset_path = '/content/drive/MyDrive/Thesis Dataset/'
data = pd.read_csv(dataset_path+'df1.csv',encoding='utf-8')
print(f'Total number of Documents: {len(data)}')

Total number of Documents: 5600


In [9]:
data = data[['text','label']]

In [10]:
data = data.rename(columns={0:'text',1:'label'})

In [11]:
total_categories = data['label'].nunique()
print("Total number of categories:", total_categories)

Total number of categories: 5


In [12]:
category_names = data['label'].unique()
print("Category names:", category_names)

Category names: ['Software Engineering' 'DBMS' 'Data Communication' 'Cyber Security '
 'Operating System']


In [13]:
# Replace 'Cyber Security' and 'Cyber Security ' with 'Cyber Security'
data['label'] = data['label'].replace(['Cyber Security ', 'Cyber Security  '], 'Cyber Security')

# Replace 'Operating System' and 'operating System' with 'Operating System'
data['label'] = data['label'].replace(['Operating System', 'operating System'], 'Operating System')

In [14]:
unique_classes = data['label'].unique()
print("Unique classes:", unique_classes)

Unique classes: ['Software Engineering' 'DBMS' 'Data Communication' 'Cyber Security'
 'Operating System']


In [15]:
print("First rows of dataset")
data.head()

First rows of dataset


,text,label
0,সফটওয়্যার ইঞ্জিনিয়ারিং দুইটি শব্দ সফটওয়্যার...,Software Engineering
1,সফটওয়্যার কিছু প্রোগ্রামের সমষ্টি,Software Engineering
2,সমস্ত প্রোগ্রাম যখন কোন নির্ধারিত সমস্যা সমাধা...,Software Engineering
3,অপরদিকে ইঞ্জিনিয়ারিং বলতে যে সকল নীতিমালা ও ম...,Software Engineering
4,সফটওয়্যার ইঞ্জিনিয়ারিং বলতে কী বুঝায়,Software Engineering


In [16]:
def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\u00C0-\u017F"          #latin
                           u"\u2000-\u206F"          #generalPunctuations

                           "]+", flags=re.UNICODE)
    #english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    #latin_pattern=re.compile('[A-Za-z\u00C0-\u00D6\u00D8-\u00f6\u00f8-\u00ff\s]*',)

    text=emoji_pattern.sub(r'', text)
    #text=english_pattern.sub(r'', text)
    text = re.sub(r'₀|₁|₂|₃|₄|₅|₆|₇|₈|₉', '', text)
    #text = re.sub(r'⁰|¹|²|³|⁴|⁵|⁶|⁷|⁸|⁹|⁻|⁺|ⁿ|ᵗ', '', text)

    return text

def remove_punctuations(my_str):
    # define punctuation
    punctuations = '''````£|¢|Ñ+-*/=৳০১২৩৪৫৬৭৮৯012–34567•89।!()-−[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰θ→°π∈∝α≤≥Δβε≠�￰৷￰'''

    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct



def joining(text):
    out=' '.join(text)
    return out

def preprocessing(text):
    out=remove_punctuations(replace_strings(text))
    return out


In [17]:
data['cleaned'] = data['text'].apply(lambda x: preprocessing(str(x)))

In [18]:
sample_data = range(4000, 5000)
for i in sample_data:
    print(data['cleaned'][i])

গ্রে হ্যাট হ্যাকাররা  আবার তা নিজের স্বার্থের জন্যও ব্যবহার করতে পারে
ব্ল্যাক হ্যাট হ্যাকার Black Hat Hacker এই ধরনের হ্যাকাররা সবচেয়ে ভয়ংকর প্রকৃতির হ্যাকার 
ব্ল্যাক হ্যাট হ্যাকার  কোনো একটি সিকিউরিটি সিস্টেমের ত্রুটিগুলো বের করলে দ্রুত ঐ ত্রুটিকে নিজের স্বার্থে কাজে লাগায় 
ব্ল্যাক হ্যাট হ্যাকার  এরা সিস্টেম নষ্ট করে বিভিন্ন ভাইরাস ছড়িয়ে দেয় 
ব্ল্যাক হ্যাট হ্যাকার  তারা ভবিষ্যতে নিজে আবার যেন ঢুকতে পারে সে পথ রাখে 
ব্ল্যাক হ্যাট হ্যাকার  সর্বোপরি ঐ সিস্টেমের অধীনে যেসকল সাবসিস্টেম রয়েছে সেগুলোতেও ঢুকতে চেষ্টা করে
ব্লাক হ্যাট হ্যাকারদের কারণেই সবচেয়ে বেশি সাইবার ক্রাইম হয় এবং ইন্টারনেটে ধবংসাত্মক ভাইরাস এরাই তৈরি করে থাকে
ক্রাক্যাররাও  একধরনের হ্যাকার তবে এরা বিভিন্ন সিস্টেমে ঢুকে সেটির অনেক কিছু পরিবর্তন করে এবং কম্পিউটার ভাইরাস দিয়ে সকল তথ্য মুছে দেয় 
 অনেক সময় বিভিন্ন ওয়েবসাইট হ্যাক করে তার বিনিময়ে টাকা দাবি করা হয় এ ধরনের হ্যাকারকে ক্র্যাকার বলা হয়
পৃথিবীর বিভিন্ন দেশে যেসব সাইবার আইন করা হয়েছে তা মূলত এসব ক্র্যাকারদের জন্যই করা হয়েছে 
ক্র্যাকাররা বিভিন্নভাবে কাজ করে থাকে
তারা অনেক

In [19]:
stop =["অতএব","অথচ","অথবা","অনুযায়ী","অনেক","অনেকে","অনেকেই","অন্তত","অন্য","অবধি","অবশ্য","অর্থাত","আই","আগামী","আগে","আগেই","আছে","আজ","আদ্যভাগে","আপনার","আপনি","আবার","আমরা","আমাকে","আমাদের","আমার","আমি","আর","আরও","ই","ইত্যাদি","ইহা","উচিত","উত্তর","উনি","উপর","উপরে","এ","এঁদের","এঁরা","এই","একই","একটি","একবার","একে","এক্","এখন","এখনও","এখানে","এখানেই","এটা","এটাই","এটি","এত","এতটাই","এতে","এদের","এব","এবং","এবার","এমন","এমনকী","এমনি","এর","এরা","এল","এস","এসে","ঐ","ও","ওঁদের","ওঁর","ওঁরা","ওই","ওকে","ওখানে","ওদের","ওর","ওরা","কখনও","কত","কবে","কমনে","কয়েক","কয়েকটি","করছে","করছেন","করতে","করবে","করবেন","করলে","করলেন","করা","করাই","করায়","করার","করি","করিতে","করিয়া","করিয়ে","করে","করেই","করেছিলেন","করেছে","করেছেন","করেন","কাউকে","কাছ","কাছে","কাজ","কাজে","কারও","কারণ","কি","কিংবা","কিছু","কিছুই","কিন্তু","কী","কে","কেউ","কেউই","কেখা","কেন","কোটি","কোন","কোনও","কোনো","ক্ষেত্রে","কয়েক","খুব","গিয়ে","গিয়েছে","গিয়ে","গুলি","গেছে","গেল","গেলে","গোটা","চলে","চান","চায়","চার","চালু","চেয়ে","চেষ্টা","ছাড়া","ছাড়াও","ছিল","ছিলেন","জন","জনকে","জনের","জন্য","জন্যওজে","জানতে","জানা","জানানো","জানায়","জানিয়ে","জানিয়েছে","জে","জ্নজন","টি","ঠিক","তখন","তত","তথা","তবু","তবে","তা","তাঁকে","তাঁদের","তাঁর","তাঁরা","তাঁাহারা","তাই","তাও","তাকে","তাতে","তাদের","তার","তারপর","তারা","তারৈ","তাহলে","তাহা","তাহাতে","তাহার","তিনঐ","তিনি","তিনিও","তুমি","তুলে","তেমন","তো","তোমার","থাকবে","থাকবেন","থাকা","থাকায়","থাকে","থাকেন","থেকে","থেকেই","থেকেও","দিকে","দিতে","দিন","দিয়ে","দিয়েছে","দিয়েছেন","দিলেন","দু","দুই","দুটি","দুটো","দেওয়া","দেওয়ার","দেওয়া","দেখতে","দেখা","দেখে","দেন","দেয়","দ্বারা","ধরা","ধরে","ধামার","নতুন","নয়","না","নাই","নাকি","নাগাদ","নানা","নিজে","নিজেই","নিজেদের","নিজের","নিতে","নিয়ে","নিয়ে","নেই","নেওয়া","নেওয়ার","নেওয়া","নয়","পক্ষে","পর","পরে","পরেই","পরেও","পর্যন্ত","পাওয়া","পাচ","পারি","পারে","পারেন","পি","পেয়ে","পেয়্র্","প্রতি","প্রথম","প্রভৃতি","প্রযন্ত","প্রাথমিক","প্রায়","প্রায়","ফলে","ফিরে","ফের","বক্তব্য","বদলে","বন","বরং","বলতে","বলল","বললেন","বলা","বলে","বলেছেন","বলেন","বসে","বহু","বা","বাদে","বার","বি","বিনা","বিভিন্ন","বিশেষ","বিষয়টি","বেশ","বেশি","ব্যবহার","ব্যাপারে","ভাবে","ভাবেই","মতো","মতোই","মধ্যভাগে","মধ্যে","মধ্যেই","মধ্যেও","মনে","মাত্র","মাধ্যমে","মোট","মোটেই","যখন","যত","যতটা","যথেষ্ট","যদি","যদিও","যা","যাঁর","যাঁরা","যাওয়া","যাওয়ার","যাওয়া","যাকে","যাচ্ছে","যাতে","যাদের","যান","যাবে","যায়","যার","যারা","যিনি","যে","যেখানে","যেতে","যেন","যেমন","র","রকম","রয়েছে","রাখা","রেখে","লক্ষ","শুধু","শুরু","সঙ্গে","সঙ্গেও","সব","সবার","সমস্ত","সম্প্রতি","সহ","সহিত","সাথে","সাধারণ","সামনে","সি","সুতরাং","সে","সেই","সেখান","সেখানে","সেটা","সেটাই","সেটাও","সেটি","স্পষ্ট","স্বয়ং","হইতে","হইবে","হইয়া","হওয়া","হওয়ায়","হওয়ার","হচ্ছে","হত","হতে","হতেই","হন","হবে","হবেন","হয়","হয়তো","হয়নি","হয়ে","হয়েই","হয়েছিল","হয়েছে","হয়েছেন","হল","হলে","হলেই","হলেও","হলো","হাজার","হিসাবে","হৈলে","হোক","হয়"]
display(stop)

['অতএব',
 'অথচ',
 'অথবা',
 'অনুযায়ী',
 'অনেক',
 'অনেকে',
 'অনেকেই',
 'অন্তত',
 'অন্য',
 'অবধি',
 'অবশ্য',
 'অর্থাত',
 'আই',
 'আগামী',
 'আগে',
 'আগেই',
 'আছে',
 'আজ',
 'আদ্যভাগে',
 'আপনার',
 'আপনি',
 'আবার',
 'আমরা',
 'আমাকে',
 'আমাদের',
 'আমার',
 'আমি',
 'আর',
 'আরও',
 'ই',
 'ইত্যাদি',
 'ইহা',
 'উচিত',
 'উত্তর',
 'উনি',
 'উপর',
 'উপরে',
 'এ',
 'এঁদের',
 'এঁরা',
 'এই',
 'একই',
 'একটি',
 'একবার',
 'একে',
 'এক্',
 'এখন',
 'এখনও',
 'এখানে',
 'এখানেই',
 'এটা',
 'এটাই',
 'এটি',
 'এত',
 'এতটাই',
 'এতে',
 'এদের',
 'এব',
 'এবং',
 'এবার',
 'এমন',
 'এমনকী',
 'এমনি',
 'এর',
 'এরা',
 'এল',
 'এস',
 'এসে',
 'ঐ',
 'ও',
 'ওঁদের',
 'ওঁর',
 'ওঁরা',
 'ওই',
 'ওকে',
 'ওখানে',
 'ওদের',
 'ওর',
 'ওরা',
 'কখনও',
 'কত',
 'কবে',
 'কমনে',
 'কয়েক',
 'কয়েকটি',
 'করছে',
 'করছেন',
 'করতে',
 'করবে',
 'করবেন',
 'করলে',
 'করলেন',
 'করা',
 'করাই',
 'করায়',
 'করার',
 'করি',
 'করিতে',
 'করিয়া',
 'করিয়ে',
 'করে',
 'করেই',
 'করেছিলেন',
 'করেছে',
 'করেছেন',
 'করেন',
 'কাউকে',
 'কাছ',
 'কাছে',
 'কাজ',
 'কাজে',
 'কারও',
 '

In [20]:
def stopwordRemoval(text):
    x=str(text)
    l=x.split()

    stm=[elem for elem in l if elem not in stop]

    out=' '.join(stm)

    return str(out)

In [21]:
# Apply the stopwordRemoval function to the 'clean text' column
data['cleaned'] = data['cleaned'].apply(lambda x: stopwordRemoval(str(x)))

# Iterate through the first 1000 rows and print the 'clean text' column values
sample_data = range(0, 1000)
for i in sample_data:
    print(data['cleaned'][i])

সফটওয়্যার ইঞ্জিনিয়ারিং দুইটি শব্দ সফটওয়্যার ইঞ্জিনিয়ারিংএর সমন্বয়ে গঠিত
সফটওয়্যার প্রোগ্রামের সমষ্টি
প্রোগ্রাম নির্ধারিত সমস্যা সমাধানের উদ্দেশ্যে তৈরি সফটওয়্যার প্রোডাক্ট
অপরদিকে ইঞ্জিনিয়ারিং সকল নীতিমালা মেথড সমূহ প্রোডাক্টটি Software ফিনিশিং প্রোডাক্ট সেগুলোর সুশৃঙ্খল প্রয়োগ করাকে বুঝায়
সফটওয়্যার ইঞ্জিনিয়ারিং বুঝায়
সফটওয়্যার ইঞ্জিনিয়ারিং ইঞ্জিনিয়ারিং শাখা সফটওয়্যার প্রডাক্ট ডেভেলপমেন্টের সুনির্ধারিত বৈজ্ঞানিক নীতিমালা Scientific principle মেথড ব্যবহৃত প্রক্রিয়া Procedures সম্পর্কিত
সফটওয়্যার ইঞ্জিনিয়ারিংএর outcome দক্ষ Efficient নির্ভরযোগ্য Reliable সফটওয়্যার প্রোডাক্ট
অর্থাৎ ডেভেলপমেন্ট অপারেশন সফটওয়্যার সংরক্ষণের সুষম শৃঙ্খলাবদ্ধ quantifiable পদ্ধতির সফল প্রয়োগই সফটওয়্যার ইঞ্জিনিয়ারিং
জার্মান কম্পিউটার বিজ্ঞানী Fritz Bauer মতে প্রকৃত মেশিনসমূহে কার্যকরিভাবে সাউন্ড ইঞ্জিনিয়ারিং Core Engineeringএর নীতিসমূহ সফটওয়্যার সমূহে সুষ্ঠ ব্যবহারই সফটওয়্যার ইঞ্জিনিয়ারিং
পরিশেষে পূর্ণ সফটওয়্যার ডিজাইন ডেভেলপমেন্ট রক্ষণাবেক্ষণের ইঞ্জিনিয়ারিং শাখা সফটওয়্যার ইঞ্জিনিয়ার

In [22]:
data['label'] = data['label'].replace({'Software Engineering':0, 'DBMS':1, 'Data Communication':2 ,'Cyber Security':3,
 'Operating System':4})
print("First rows of dataset")
data.head()

First rows of dataset


,text,label,cleaned
0,সফটওয়্যার ইঞ্জিনিয়ারিং দুইটি শব্দ সফটওয়্যার...,0,সফটওয়্যার ইঞ্জিনিয়ারিং দুইটি শব্দ সফটওয়্যার...
1,সফটওয়্যার কিছু প্রোগ্রামের সমষ্টি,0,সফটওয়্যার প্রোগ্রামের সমষ্টি
2,সমস্ত প্রোগ্রাম যখন কোন নির্ধারিত সমস্যা সমাধা...,0,প্রোগ্রাম নির্ধারিত সমস্যা সমাধানের উদ্দেশ্যে ...
3,অপরদিকে ইঞ্জিনিয়ারিং বলতে যে সকল নীতিমালা ও ম...,0,অপরদিকে ইঞ্জিনিয়ারিং সকল নীতিমালা মেথড সমূহ প...
4,সফটওয়্যার ইঞ্জিনিয়ারিং বলতে কী বুঝায়,0,সফটওয়্যার ইঞ্জিনিয়ারিং বুঝায়


In [23]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

# Print the shapes of the resulting datasets
print("Training data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)
print("Testing data shape:", test_data.shape)

Training data shape: (4480, 3)
Validation data shape: (560, 3)
Testing data shape: (560, 3)


In [24]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [25]:
from urllib.request import urlopen
import gzip

# get the vectors
file = gzip.open(urlopen('https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bn.300.vec.gz'))

In [26]:
vocab_and_vectors = {}
# put words as dict indexes and vectors as words values
for line in file:
  values = line.split()
  word = values [0].decode('utf-8')
  vector = np.asarray(values[1:], dtype='float32')
  vocab_and_vectors[word] = vector

In [27]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['cleaned'])

In [28]:
word_index = tokenizer.word_index
vocab_size = len(word_index )+1
print(word_index)
print(vocab_size)

{'ডাটা': 1, 'যায়': 2, 'সিস্টেম': 3, 'সফটওয়্যার': 4, 'ডাটাবেস': 5, 'কম্পিউটার': 6, 'ব্যবহৃত': 7, 'ধরনের': 8, 'তথ্য': 9, 'সিস্টেমের': 10, 'নেটওয়ার্ক': 11, 'তৈরি': 12, 'সিস্টেমে': 13, 'প্রয়োজন': 14, 'হয়ে': 15, 'সকল': 16, 'সাইবার': 17, 'ডিজাইন': 18, 'অপারেটিং': 19, 'প্রোগ্রাম': 20, 'এক': 21, 'ফাইল': 22, 'নিরাপত্তা': 23, 'সময়': 24, 'প্রদান': 25, 'এনটিটি': 26, 'একাধিক': 27, 'প্রয়োজনীয়': 28, 'নেটওয়ার্কের': 29, 'ইউজার': 30, 'প্রসেস': 31, 'নির্দিষ্ট': 32, 'system': 33, 'সার্ভার': 34, 'সিগন্যাল': 35, 'সাধারণত': 36, 'ডিভাইস': 37, 'যোগাযোগ': 38, 'টেস্টিং': 39, 'পরিবর্তন': 40, 'কারণে': 41, 'ডেটা': 42, 'পদ্ধতি': 43, 'হিসেবে': 44, 'ব্যবস্থা': 45, 'প্রতিটি': 46, 'হয়েছে': 47, 'রয়েছে': 48, 'data': 49, 'অর্থাৎ': 50, 'সংযুক্ত': 51, 'নেটওয়ার্কে': 52, 'মেমোরি': 53, 'গুরুত্বপূর্ণ': 54, 'সম্ভব': 55, 'দিয়ে': 56, 'কম্পিউটারের': 57, 'ডাটাবেসের': 58, 'কম্পিউটারে': 59, 'সমূহ': 60, 'দেয়': 61, 'সম্পন্ন': 62, 'একজন': 63, 'কম': 64, 'মডেল': 65, 'প্রকার': 66, 'হ্যাকার': 67, 'ফায়ারওয়াল': 68, 'পদ্ধতিতে': 69, 'database': 7

In [29]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in word_index.items():
  embedding_vector = vocab_and_vectors.get(word)
  # words that cannot be found will be set to 0
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [30]:
max_len = 40
# Training Sequences
train_sequences = tokenizer.texts_to_sequences(train_data['cleaned'])
print(len(tokenizer.word_index))
train_pad_sequences =  keras.preprocessing.sequence.pad_sequences(train_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Training Sequences :" ,train_pad_sequences.shape)

# validation Sequences
val_sequences = tokenizer.texts_to_sequences(val_data['cleaned'])
val_pad_sequences =  keras.preprocessing.sequence.pad_sequences(val_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Validation Sequences :" ,val_pad_sequences.shape)

# test Sequences
test_sequences = tokenizer.texts_to_sequences(test_data['cleaned'])
test_pad_sequences =  keras.preprocessing.sequence.pad_sequences(test_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Test Sequences :" ,test_pad_sequences.shape)

10025
Number of Training Sequences : (4480, 40)
Number of Validation Sequences : (560, 40)
Number of Test Sequences : (560, 40)


In [31]:
num_classes = 5
train_labels = tf.keras.utils.to_categorical(train_data['label'], num_classes)
val_labels = tf.keras.utils.to_categorical(val_data['label'], num_classes)
test_labels = tf.keras.utils.to_categorical(test_data['label'], num_classes)

In [32]:
# Enable CUDA support in TensorFlow
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the first GPU
        tf.config.set_visible_devices(gpus[0], 'GPU')
        # Enable memory growth to allocate GPU memory on an as-needed basis
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

##CNN WITH FAST TEXT


In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Define the improved CNN model architecture
cnn_model = Sequential()
cnn_model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], trainable=True, input_length=max_len))  # Allow embedding to be trainable
cnn_model.add(Conv1D(256, 5, activation='relu'))  # Increased filters
cnn_model.add(MaxPooling1D())
cnn_model.add(Dropout(0.3))  # Increased dropout rate
cnn_model.add(Conv1D(256, 5, activation='relu'))  # Added another Conv1D layer
cnn_model.add(MaxPooling1D())
cnn_model.add(Dropout(0.3))  # Increased dropout rate
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dropout(0.3))  # Added dropout to dense layer
cnn_model.add(Dense(num_classes, activation='softmax'))

# Compile the model
cnn_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

cnn_model.summary()

# Define early stopping
early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

# Train the model
history = cnn_model.fit(train_pad_sequences, train_labels, validation_data=(val_pad_sequences, val_labels), epochs=30, batch_size=32,
    callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = cnn_model.evaluate(test_pad_sequences, test_labels)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)

# Get the predicted labels
predictions = cnn_model.predict(test_pad_sequences)
predicted_labels = predictions.argmax(axis=1)

true_labels = np.argmax(test_labels, axis=1)

# Calculate evaluation metrics
test_precision = precision_score(true_labels, predicted_labels, average='weighted')
test_recall = recall_score(true_labels, predicted_labels, average='weighted')
test_f1 = f1_score(true_labels, predicted_labels, average='weighted')
confusion_mat = confusion_matrix(true_labels, predicted_labels)

# Print the results
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)
print("Test F1 Score:", test_f1)
print("Confusion Matrix:")
print(confusion_mat)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 300)           3007800   
                                                                 
 conv1d (Conv1D)             (None, 36, 256)           384256    
                                                                 
 max_pooling1d (MaxPooling1  (None, 18, 256)           0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 18, 256)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 14, 256)           327936    
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 7, 256)            0         
 g1D)                                                   

##BILSTM

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Define the BiLSTM model architecture
bilstm_model = Sequential()
bilstm_model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], trainable=True, input_length=max_len))  # Allow embedding to be trainable
bilstm_model.add(Bidirectional(LSTM(128, return_sequences=True)))  # Bidirectional LSTM layer with 128 units
bilstm_model.add(Dropout(0.3))  # Increased dropout rate
bilstm_model.add(Bidirectional(LSTM(128)))  # Added another Bidirectional LSTM layer
bilstm_model.add(Dropout(0.3))  # Increased dropout rate
bilstm_model.add(Flatten())
bilstm_model.add(Dense(128, activation='relu'))
bilstm_model.add(Dropout(0.3))  # Added dropout to dense layer
bilstm_model.add(Dense(num_classes, activation='softmax'))

# Compile the model
bilstm_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

bilstm_model.summary()

# Define early stopping
early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

# Train the model
history = bilstm_model.fit(train_pad_sequences, train_labels, validation_data=(val_pad_sequences, val_labels), epochs=30, batch_size=32,
    callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = bilstm_model.evaluate(test_pad_sequences, test_labels)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)

# Get the predicted labels
predictions = bilstm_model.predict(test_pad_sequences)
predicted_labels = predictions.argmax(axis=1)

true_labels = np.argmax(test_labels, axis=1)

# Calculate evaluation metrics
test_precision = precision_score(true_labels, predicted_labels, average='weighted')
test_recall = recall_score(true_labels, predicted_labels, average='weighted')
test_f1 = f1_score(true_labels, predicted_labels, average='weighted')
confusion_mat = confusion_matrix(true_labels, predicted_labels)

# Print the results
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)
print("Test F1 Score:", test_f1)
print("Confusion Matrix:")
print(confusion_mat)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 40, 300)           3007800   
                                                                 
 bidirectional (Bidirection  (None, 40, 256)           439296    
 al)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 40, 256)           0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 256)               394240    
 onal)                                                           
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 flatten_1 (Flatten)         (None, 256)              

In [35]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Bidirectional, LSTM, Dropout, Flatten, Dense, BatchNormalization, Add, Input, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf

# Define the CNN+BiLSTM model architecture
input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], trainable=False)(input_layer)

# Convolutional block 1
conv1 = Conv1D(256, 5, activation='relu', padding='same')(embedding_layer)  # Use padding='same' to maintain sequence length
conv1 = BatchNormalization()(conv1)
conv1 = GlobalMaxPooling1D()(conv1)  # GlobalMaxPooling1D instead of MaxPooling1D

# Convolutional block 2
conv2 = Conv1D(256, 5, activation='relu', padding='same')(embedding_layer)  # Apply conv2 directly on embedding_layer for residual connection compatibility
conv2 = BatchNormalization()(conv2)
conv2 = GlobalMaxPooling1D()(conv2)  # GlobalMaxPooling1D instead of MaxPooling1D

# Residual connection
residual = Add()([conv1, conv2])

# Expand dimensions to make it compatible with LSTM input
expanded_residual = tf.expand_dims(residual, axis=1)

# BiLSTM block 1
bilstm1 = Bidirectional(LSTM(128, return_sequences=True))(expanded_residual)
bilstm1 = Dropout(0.5)(bilstm1)

# BiLSTM block 2
bilstm2 = Bidirectional(LSTM(128, return_sequences=True))(bilstm1)
bilstm2 = Dropout(0.5)(bilstm2)

# Flatten and Dense layers
flatten = Flatten()(bilstm2)
dense1 = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(flatten)
dense1 = Dropout(0.5)(dense1)
output_layer = Dense(num_classes, activation='softmax')(dense1)

# Compile the model
cnn_bilstm_model = Model(inputs=input_layer, outputs=output_layer)
cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

cnn_bilstm_model.summary()

# Define early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Train the model
history = cnn_bilstm_model.fit(train_pad_sequences, train_labels, validation_data=(val_pad_sequences, val_labels), epochs=30, batch_size=32,
                               callbacks=[early_stopping, reduce_lr])

# Evaluate the model on the test set
test_loss, test_accuracy = cnn_bilstm_model.evaluate(test_pad_sequences, test_labels)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)

# Get the predicted labels
predictions = cnn_bilstm_model.predict(test_pad_sequences)
predicted_labels = predictions.argmax(axis=1)

true_labels = np.argmax(test_labels, axis=1)

# Calculate evaluation metrics
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

test_precision = precision_score(true_labels, predicted_labels, average='weighted')
test_recall = recall_score(true_labels, predicted_labels, average='weighted')
test_f1 = f1_score(true_labels, predicted_labels, average='weighted')
confusion_mat = confusion_matrix(true_labels, predicted_labels)

# Print the results
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)
print("Test F1 Score:", test_f1)
print("Confusion Matrix:")
print(confusion_mat)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 40)]                 0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 40, 300)              3007800   ['input_1[0][0]']             
                                                                                                  
 conv1d_2 (Conv1D)           (None, 40, 256)              384256    ['embedding_2[0][0]']         
                                                                                                  
 conv1d_3 (Conv1D)           (None, 40, 256)              384256    ['embedding_2[0][0]']         
                                                                                              